In [40]:
import pandas as pd
import numpy as np


In [41]:
def beta(year: int, df_company: pd.DataFrame, df2: pd.DataFrame) -> float:
  dfc_year = df_company[df_company['Date'].dt.year == year].dropna()
  df2_year = df2[df2['Date'].dt.year == year].dropna()
  cor = pd.DataFrame(dfc_year['Change']).join(pd.DataFrame(df2_year['Change']), lsuffix='l', rsuffix='r', how='inner')
  corr = cor.corr().iloc[1, 0]
  std1 = cor['Changel'].std()
  std2 = cor['Changer'].std()
  return corr * (std1/std2)

def deviation(frame: pd.DataFrame, year: int) -> float:
  frame = frame[frame['Date'].dt.year == year]
  return frame['Change'].std()

def cost_of_debt(tax_rate: float, fininfo:pd.DataFrame, year: int) -> np.ndarray:
  fininfo = fininfo[fininfo.index == year]
  pretax_debt = np.abs(fininfo.iloc[:, 2]) / fininfo.iloc[:, 0] * 100
  return (pretax_debt * (1-tax_rate/100)).values[0]

def k(frame: pd.DataFrame, year: int) -> float:
  frame = frame[frame['Date'].dt.year == year]
  changes = frame['Change'].values/100 +1
  return (np.prod(changes) - 1)*100

def leverage(frame: pd.DataFrame, year: int) -> float:
  frame = frame[frame.index == year]
  return (frame.iloc[:, 0] / frame.iloc[:, 1]).values[0]

def cost_of_debt_industry(companies: list[pd.DataFrame], tax_rate: float, year: int) -> float:
  IE = []
  TD = []
  for company in companies:
    company = company[company.index == year]
    IE.append(company.iloc[:, 2].values[0])
    TD.append(company.iloc[:, 0].values[0])
  return abs(sum(IE))/sum(TD) * (1-tax_rate/100) * 100
    
def leverage_industry(companies: list[pd.DataFrame], year: int) -> float:
  Ls= []
  for company in companies:
    L = leverage(company, year)
    Ls.append(L)
  return np.array(Ls).mean()


def k_CAPM(k_entity: float, risk_free: float, betta: float) -> float:
  return risk_free + betta*(k_entity-risk_free)

def Delta(leverage: float, COD: float, tax: float, risk_free: float) -> float:
  return leverage*(1-tax/100) * (risk_free - COD)

def k_CAPM2(k_entity: float, risk_free: float, betta: float, delta:float) -> float:
  return risk_free + betta*(k_entity-risk_free) + delta

In [42]:
Market = pd.read_excel("SPX.xlsx").dropna()
Industry = pd.read_excel("2021-2024_Chip_Industry_Returns.xlsx").dropna()
TSM_YIELD = pd.read_excel("2021-2024_TSM_Returns.xlsx").dropna()
INTEL = pd.read_excel("INTEL_DEBT-TO-EQUITY.xlsx", index_col=0).dropna()
BCOM = pd.read_excel("BROADCOM_DEBT-TO-EQUITY.xlsx", index_col=0).dropna()
NVD = pd.read_excel("NVIDIA_DEBT-TO-EQUITY.xlsx", index_col=0).dropna()
SAMS = pd.read_excel("SAMSUNG_DEBT-TO-EQUITY.xlsx", index_col=0).dropna()
QCOM = pd.read_excel("QUALCOMM_DEBT-TO-EQUITY.xlsx", index_col=0).dropna()
taxes = pd.read_csv("weighted_corporate_taxes.csv", index_col=0).dropna()
TSM = pd.read_excel('tsm_financial_information.xlsx', index_col=0).dropna()
tetr = pd.read_excel('effective_tax_rates_tsmc.xlsx', index_col=0).dropna()


In [43]:
kf_2021 = 1.52
kf_2022 = 3.88 
kf_2023 = 3.88 
kf_2024 = 4.58



In [44]:
k_tsm_2021 = k(TSM_YIELD, 2021)
k_tsm_2022 = k(TSM_YIELD, 2022)
k_tsm_2023 = k(TSM_YIELD, 2023)
k_tsm_2024 = k(TSM_YIELD, 2024)
k_us_2021 = k(Market, 2021)
k_us_2022 = k(Market, 2022)
k_us_2023 = k(Market, 2023)
k_us_2024 = k(Market, 2024)
k_ind_2021 = k(Industry, 2021)
k_ind_2022 = k(Industry, 2022)
k_ind_2023 = k(Industry, 2023)
k_ind_2024 = k(Industry, 2024)
k_us_2021, k_us_2022, k_us_2023, k_us_2024


(np.float64(28.303531979240248),
 np.float64(-19.437920612458072),
 np.float64(24.233434801403696),
 np.float64(23.29969165229384))

In [45]:
k_tsm_2021, k_tsm_2022, k_tsm_2023, k_tsm_2024

(np.float64(0.2542009814207935),
 np.float64(-37.02222965383163),
 np.float64(39.607349317852304),
 np.float64(89.91614458771046))

In [46]:
k_ind_2021, k_ind_2022, k_ind_2023, k_ind_2024

(np.float64(19.42411050522004),
 np.float64(-35.805143898538425),
 np.float64(65.02112657438623),
 np.float64(19.34867097913755))

In [47]:
b_market_tsm_2021 = beta(2021, TSM_YIELD, Market)
b_market_tsm_2022 = beta(2022, TSM_YIELD, Market)
b_market_tsm_2023 = beta(2023, TSM_YIELD, Market)
b_market_tsm_2024 = beta(2024, TSM_YIELD, Market)
b_ind_tsm_2021 = beta(2021, TSM_YIELD, Industry)
b_ind_tsm_2022 = beta(2022, TSM_YIELD, Industry)
b_ind_tsm_2023 = beta(2023, TSM_YIELD, Industry)
b_ind_tsm_2024 = beta(2024, TSM_YIELD, Industry)
b_market_tsm_2021, b_market_tsm_2022, b_market_tsm_2023, b_market_tsm_2024

(np.float64(0.3240734793326558),
 np.float64(1.2472246577199553),
 np.float64(1.5453811905848058),
 np.float64(0.29938433456211466))

In [48]:
b_ind_tsm_2021, b_ind_tsm_2022, b_ind_tsm_2023, b_ind_tsm_2024

(np.float64(0.593520890683248),
 np.float64(0.900943911800595),
 np.float64(1.0241834239090033),
 np.float64(0.8140325120277278))

In [49]:
std_tsm_2021 = deviation(TSM_YIELD, 2021)
std_tsm_2022 = deviation(TSM_YIELD, 2022)
std_tsm_2023 = deviation(TSM_YIELD, 2023)
std_tsm_2024 = deviation(TSM_YIELD, 2024)
std_us_2021 = deviation(Market, 2021)
std_us_2022 = deviation(Market, 2022)
std_us_2023 = deviation(Market, 2023)
std_us_2024 = deviation(Market, 2024)
std_ind_2021 = deviation(Industry, 2021)
std_ind_2022 = deviation(Industry, 2022)
std_ind_2023 = deviation(Industry, 2023)
std_ind_2024 = deviation(Industry, 2024)

In [50]:
L_tsm_2021 = leverage(TSM, 2021)
L_tsm_2022 = leverage(TSM, 2022)
L_tsm_2023 = leverage(TSM, 2023)
L_tsm_2024 = leverage(TSM, 2024)
L_tsm_2021, L_tsm_2022, L_tsm_2023, L_tsm_2024

(np.float64(0.3512682862670756),
 np.float64(0.30528796723046425),
 np.float64(0.2855937996454409),
 np.float64(0.24539010373727121))

In [51]:
CD_tsm_2021 = cost_of_debt(taxes.loc['United States'].iloc[-1], TSM, 2021)
CD_tsm_2022 = cost_of_debt(taxes.loc['United States'].iloc[-2], TSM, 2022)
CD_tsm_2023 = cost_of_debt(taxes.loc['United States'].iloc[-3], TSM, 2023)
CD_tsm_2024 = cost_of_debt(taxes.loc['United States'].iloc[-4], TSM, 2024)
CD_tsm_2021, CD_tsm_2022, CD_tsm_2023, CD_tsm_2024

(np.float64(0.5318601832716341),
 np.float64(0.9786181533242252),
 np.float64(0.902963362844598),
 np.float64(0.7432814394181075))

In [52]:
CD_ind_2021 = cost_of_debt_industry([TSM, INTEL, BCOM, NVD, SAMS, QCOM], taxes.loc['United States'].iloc[-1] , 2021)
CD_ind_2022 = cost_of_debt_industry([TSM, INTEL, BCOM, NVD, SAMS, QCOM], taxes.loc['United States'].iloc[-2] , 2022)
CD_ind_2023 = cost_of_debt_industry([TSM, INTEL, BCOM, NVD, SAMS, QCOM], taxes.loc['United States'].iloc[-3] , 2023)
CD_ind_2024 = cost_of_debt_industry([TSM, INTEL, BCOM, NVD, SAMS, QCOM], taxes.loc['United States'].iloc[-4] , 2024)
CD_ind_2021, CD_ind_2022, CD_ind_2023, CD_ind_2024


(np.float64(1.9657900983399792),
 np.float64(1.9659749191467684),
 np.float64(2.1073270593734326),
 np.float64(2.652455024539622))

In [53]:
L_ind_2021 = leverage_industry([TSM, INTEL, BCOM, NVD, SAMS, QCOM], 2021)
L_ind_2022 = leverage_industry([TSM, INTEL, BCOM, NVD, SAMS, QCOM], 2022)
L_ind_2023 = leverage_industry([TSM, INTEL, BCOM, NVD, SAMS, QCOM], 2023)
L_ind_2024 = leverage_industry([TSM, INTEL, BCOM, NVD, SAMS, QCOM], 2024)
L_ind_2021, L_ind_2022, L_ind_2023, L_ind_2024


(np.float64(0.7518538852987499),
 np.float64(0.6406266139993696),
 np.float64(0.619016544548768),
 np.float64(0.4399021536878209))

In [54]:
D_2021 = Delta(L_tsm_2021, CD_tsm_2021, taxes.loc['United States'].iloc[-1], 1.52)
D_2022 = Delta(L_tsm_2022, CD_tsm_2022, taxes.loc['United States'].iloc[-2], 3.88)
D_2023 = Delta(L_tsm_2023, CD_tsm_2023, taxes.loc['United States'].iloc[-3], 3.88)
D_2024 = Delta(L_tsm_2024, CD_tsm_2024, taxes.loc['United States'].iloc[-4], 4.58)
D_2021, D_2022, D_2023, D_2024

(np.float64(0.2577233686607182),
 np.float64(0.657143093172602),
 np.float64(0.6311206849890282),
 np.float64(0.7001881697706994))

In [55]:
ki_CAPM_market_2021 = k_CAPM(k_us_2021, 1.52, b_market_tsm_2021)
ki_CAPM_market_2022 = k_CAPM(k_us_2022, 3.88, b_market_tsm_2022)
ki_CAPM_market_2023 = k_CAPM(k_us_2023, 3.88, b_market_tsm_2023)
ki_CAPM_market_2024 = k_CAPM(k_us_2024, 4.58, b_market_tsm_2024)
ki_CAPM_ind_2021 = k_CAPM(k_ind_2021, 1.52, b_ind_tsm_2021)
ki_CAPM_ind_2022 = k_CAPM(k_ind_2022, 3.88, b_ind_tsm_2022)
ki_CAPM_ind_2023 = k_CAPM(k_ind_2023, 3.88, b_ind_tsm_2023)
ki_CAPM_ind_2024 = k_CAPM(k_ind_2024, 4.58, b_ind_tsm_2024)
ki_CAPM_market_2021, ki_CAPM_market_2022, ki_CAPM_market_2023, ki_CAPM_market_2024

(np.float64(10.199832397329839),
 np.float64(-25.20268555461411),
 np.float64(35.33381530588347),
 np.float64(10.184382428529965))

In [56]:
ki_CAPM_ind_2021, ki_CAPM_ind_2022, ki_CAPM_ind_2023, ki_CAPM_ind_2024

(np.float64(12.146463613949495),
 np.float64(-31.874088784318726),
 np.float64(66.49972835660863),
 np.float64(16.602178336458344))

In [57]:
ki_CAPM2_market_2021 = k_CAPM2(k_us_2021, 1.52, b_market_tsm_2021, D_2021)
ki_CAPM2_market_2022 = k_CAPM2(k_us_2022, 3.88, b_market_tsm_2022, D_2022)
ki_CAPM2_market_2023 = k_CAPM2(k_us_2023, 3.88, b_market_tsm_2023, D_2023)
ki_CAPM2_market_2024 = k_CAPM2(k_us_2024, 4.58, b_market_tsm_2024, D_2024)
ki_CAPM2_ind_2021 = k_CAPM2(k_ind_2021, 1.52, b_ind_tsm_2021, D_2021)
ki_CAPM2_ind_2022 = k_CAPM2(k_ind_2022, 3.88, b_ind_tsm_2022, D_2022)
ki_CAPM2_ind_2023 = k_CAPM2(k_ind_2023, 3.88, b_ind_tsm_2023, D_2023)
ki_CAPM2_ind_2024 = k_CAPM2(k_ind_2024, 4.58, b_ind_tsm_2024, D_2024)
ki_CAPM2_market_2021, ki_CAPM2_market_2022, ki_CAPM2_market_2023, ki_CAPM2_market_2024

(np.float64(10.457555765990557),
 np.float64(-24.545542461441507),
 np.float64(35.9649359908725),
 np.float64(10.884570598300664))

In [58]:
ki_CAPM2_ind_2021, ki_CAPM2_ind_2022, ki_CAPM2_ind_2023, ki_CAPM2_ind_2024

(np.float64(12.404186982610213),
 np.float64(-31.216945691146122),
 np.float64(67.13084904159766),
 np.float64(17.302366506229042))